In [1]:
import os
import findspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
import pandas as pd    
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64/"
os.environ["SPARK_HOME"] = "/workspace/tripx/MCS/big_data/spark-3.1.1-bin-hadoop3.2"
findspark.init()
import time 
import logging
import seaborn as sns
from pyspark.ml.linalg import Vectors

import matplotlib.pyplot as plt
import pyspark.sql.functions as f
from pyspark.sql.functions import lit
from pyspark.sql.functions import col, count, explode, sum as sum_


In [2]:
# Stop spark if it existed
SparkContext.setSystemProperty('spark.executor.memory', '200g')

try:
    sc.stop()
except:
    print('sc have not yet created!')
sc = SparkContext(master = "local[*]", appName = "Multimodal Single Cell")
# sc = SparkContext.getOrCreate()
# Init session
print("Init session")
my_spark = SparkSession.builder.getOrCreate()
print(my_spark.catalog.listTables())

23/07/27 12:49:05 WARN Utils: Your hostname, lambda-server resolves to a loopback address: 127.0.1.1; using 192.168.1.252 instead (on interface eno1)
23/07/27 12:49:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/07/27 12:49:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


sc have not yet created!
Init session
[]


In [3]:
my_spark.conf.set('spark.rapids.sql.enabled','true')


In [4]:
sc._conf.getAll()

[('spark.executor.id', 'driver'),
 ('spark.driver.host', '192.168.1.252'),
 ('spark.app.name', 'Multimodal Single Cell'),
 ('spark.driver.port', '38709'),
 ('spark.executor.memory', '200g'),
 ('spark.app.startTime', '1690436946140'),
 ('spark.app.id', 'local-1690436946687'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.sql.warehouse.dir',
  'file:/workspace/tripx/MCS/big_data/spark-warehouse/'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true')]

In [5]:
# train_cite_inputs_path = "/workspace/tripx/MCS/big_data/data/sub_sample_train_cite_inputs.csv"
# train_cite_inputs_data = my_spark.read.csv(train_cite_inputs_path,  header=True, maxColumns=30000)
# # train_cite_inputs_data.printSchema()

In [6]:
train_cite_target_path = "/workspace/tripx/MCS/big_data/data/train_cite_targets.csv"
train_cite_target_data = my_spark.read.csv(train_cite_target_path,  header=True)
# train_cite_target_data.printSchema()

In [7]:
# test_cite_inputs_path = "/workspace/tripx/MCS/big_data/data/sub_sample_test_cite_inputs.csv"
# test_cite_inputs_data = my_spark.read.csv(test_cite_inputs_path,  header=True,  maxColumns=30000)
# # test_cite_inputs_data.printSchema()

In [8]:
train_cite_target_data.createOrReplaceTempView('train_cite_target_data')

23/07/27 12:49:10 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [9]:
train_output = train_cite_target_data

In [10]:
new_cols=(column.replace('.', '_') for column in train_output.columns)
train_output = train_output.toDF(*new_cols)

In [11]:
from tqdm import tqdm

In [12]:
for col_name in tqdm(train_output.columns[1:]):
    # print(col_name)
    train_output = train_output.withColumn(col_name,  col(col_name).cast('float'))

100%|██████████| 140/140 [00:02<00:00, 50.64it/s]


## Load extraced features

In [13]:
def zscore(x):
    x_zscore = []
    for i in range(x.shape[0]):
        x_row = x[i]
        x_row = (x_row - np.mean(x_row)) / np.std(x_row)
        x_zscore.append(x_row)
    x_std = np.array(x_zscore)    
    return x_std

In [14]:
feature_path='/dataset/NeurIPS2022/2nd-solution/kaggle/src_top2/senkin13/features/'

In [15]:
train_df = pd.read_feather(feature_path+'train_cite_inputs_id.feather')
test_df = pd.read_feather(feature_path+'test_cite_inputs_id.feather')

In [16]:
import numpy as np
cite_inputs_svd_clr = np.load(feature_path+'cite_inputs_svd_clr_200.npy')
train_cite_svd_clr = cite_inputs_svd_clr[:len(train_df)]
test_cite_svd_clr = cite_inputs_svd_clr[len(train_df):]
train_cite_svd_clr = zscore(train_cite_svd_clr)
test_cite_svd_clr = zscore(test_cite_svd_clr)

In [17]:
train_cite_svd_clr.shape

(70988, 200)

In [18]:
test_cite_svd_clr.shape

(48203, 200)

## Train

In [19]:
train_feature_df = map(lambda x: (int(1), Vectors.dense(x)), train_cite_svd_clr)
train_feature_df = my_spark.createDataFrame(train_feature_df,schema=["label", "features"])

In [20]:
train_feature_df.printSchema()

root
 |-- label: long (nullable = true)
 |-- features: vector (nullable = true)



In [21]:
from pyspark.sql.functions import monotonically_increasing_id 


In [22]:
train_feature_df = train_feature_df.select("*").withColumn("id", monotonically_increasing_id())

In [23]:
# train_feature_df.show(5)

In [24]:
train_output.show(5)

+------------+------------+-----------+-----------+-----------+---------+---------+-----------+-----------+-----------+------------+-----------+------------+----------+-------------+---------+-----------+----------+-----------+------------+------------+------------+---------+-----------+------------+---------+-----------+-----------+------------+-----------+-----------+-----------+-----------+------------+------------+----------+-----------+----------+---------+----------+-----------+-----------+----------+------------+----------+-----------+-----------+----------+-----------+------------+-----------+-----------+----------+-----------+-----------+----------+----------+------------+---------+-----------+-----------+-----------+-----------+----------+----------+-----------+-----------+-----------+-----------+---------+----------+-----------+-----------+-----------+---------+-----------+---------+----------+-----------+-----------+----------+----------+-----------+-----------+------------

In [25]:
train_feature_df.show(3)

23/07/27 12:49:22 WARN TaskSetManager: Stage 2 contains a task of very large size (1314 KiB). The maximum recommended task size is 1000 KiB.


+-----+--------------------+---+
|label|            features| id|
+-----+--------------------+---+
|    1|[13.2822656631469...|  0|
|    1|[13.0130052566528...|  1|
|    1|[13.0470809936523...|  2|
+-----+--------------------+---+
only showing top 3 rows



In [26]:
train_output =  train_output.select("*").withColumn("id", monotonically_increasing_id())

In [27]:
# final_metadataset = metadataset.join(cell_type_data,metadataset.cell_id ==  cell_type_data.cell_id,"left")

In [28]:
train_data = train_feature_df.join(train_output, train_feature_df.id == train_output.id, 'left')

In [29]:
train_data

DataFrame[label: bigint, features: vector, id: bigint, cell_id: string, CD86: float, CD274: float, CD270: float, CD155: float, CD112: float, CD47: float, CD48: float, CD40: float, CD154: float, CD52: float, CD3: float, CD8: float, CD56: float, CD19: float, CD33: float, CD11c: float, HLA-A-B-C: float, CD45RA: float, CD123: float, CD7: float, CD105: float, CD49f: float, CD194: float, CD4: float, CD44: float, CD14: float, CD16: float, CD25: float, CD45RO: float, CD279: float, TIGIT: float, Mouse-IgG1: float, Mouse-IgG2a: float, Mouse-IgG2b: float, Rat-IgG2b: float, CD20: float, CD335: float, CD31: float, Podoplanin: float, CD146: float, IgM: float, CD5: float, CD195: float, CD32: float, CD196: float, CD185: float, CD103: float, CD69: float, CD62L: float, CD161: float, CD152: float, CD223: float, KLRG1: float, CD27: float, CD107a: float, CD95: float, CD134: float, HLA-DR: float, CD1c: float, CD11b: float, CD64: float, CD141: float, CD1d: float, CD314: float, CD35: float, CD57: float, CD272

In [30]:
train_data = train_data.drop('label')

In [36]:
train_data.select("features").show(3)

+--------------------+
|            features|
+--------------------+
|[13.2822656631469...|
|[13.0130052566528...|
|[13.0470809936523...|
+--------------------+
only showing top 3 rows



23/07/27 12:51:26 WARN TaskSetManager: Stage 10 contains a task of very large size (1314 KiB). The maximum recommended task size is 1000 KiB.


In [32]:
output_columns = train_output.columns[1:]

In [33]:
def train(lr_model, train_data, column_name):
    data_vector = train_data.select("features", column_name)
    data_vector = data_vector.withColumnRenamed(column_name, "label")
    # data_vector.show(5)
    # input()
    lr_model = lr.fit(data_vector)
    print("Coefficients: " + str(lr_model.coefficients))
    print("Intercept: " + str(lr_model.intercept))
    trainingSummary = lr_model.summary
    print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
    print("r2: %f" % trainingSummary.r2)

In [34]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='label', maxIter=10, regParam=0.3, elasticNetParam=0.8)

In [35]:
# for column_name in output_columns:
#     train(lr, train_data, column_name)

23/07/27 12:49:25 WARN TaskSetManager: Stage 5 contains a task of very large size (1314 KiB). The maximum recommended task size is 1000 KiB.
23/07/27 12:49:25 WARN TaskSetManager: Stage 7 contains a task of very large size (1314 KiB). The maximum recommended task size is 1000 KiB.
23/07/27 12:49:26 ERROR Executor: Exception in task 34.0 in stage 7.0 (TID 116)]
scala.MatchError: [null,1.0,[13.2056245803833,1.9891637563705444,-1.3573007583618164,-1.998184084892273,1.402286171913147,-1.4157570600509644,0.051033519208431244,-0.49477776885032654,0.0682978704571724,-1.3771345615386963,0.6391332149505615,0.9742205142974854,1.0467541217803955,-0.7432401776313782,0.06779025495052338,0.38911154866218567,0.23163321614265442,0.22612887620925903,0.5446882247924805,0.1375420093536377,-0.25348860025405884,-0.6393301486968994,-0.09563314914703369,0.5564044117927551,0.04697366803884506,-0.10778223723173141,-0.04074195772409439,0.45073267817497253,0.30734193325042725,-0.1890058070421219,-0.2705677151679

Py4JJavaError: An error occurred while calling o520.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 94 in stage 7.0 failed 1 times, most recent failure: Lost task 94.0 in stage 7.0 (TID 176) (192.168.1.252 executor driver): scala.MatchError: [null,1.0,[13.504267692565918,-0.6677440404891968,-1.8945854902267456,-1.5007598400115967,0.3464559316635132,-0.73587566614151,0.9236093759536743,0.668449342250824,-1.0647611618041992,0.49758464097976685,-0.5393695831298828,0.6932887434959412,0.18159019947052002,0.09574350714683533,-1.084779143333435,-0.7369211912155151,-0.5531873106956482,-0.4570564329624176,-0.35425737500190735,0.1832999736070633,-0.6712433099746704,-0.5441982746124268,0.3688693344593048,-0.06870487332344055,-0.11058980971574783,0.18056589365005493,0.0400826632976532,-0.3141237199306488,0.18488915264606476,-0.1900169551372528,0.04330188035964966,-0.14515717327594757,0.3441285490989685,0.09944163262844086,-0.16283781826496124,0.4030168354511261,-0.13

## Test

In [ ]:
test_feature_df = map(lambda x: (int(1), Vectors.dense(x)), test_cite_svd_clr)
test_feature_df = my_spark.createDataFrame(test_feature_df,schema=["label", "features"])

In [ ]:
test_feature_df.printSchema()

root
 |-- label: long (nullable = true)
 |-- features: vector (nullable = true)

